# Data preparation
* Download data from kaggle upload it to google collab workspace
* kaggle Dataset link: https://www.kaggle.com/datasets/abtabm/multiclassimagedatasetairplanecar/
* Extract the zip file

In [ ]:
import zipfile
import os

# Path to your ZIP file
zip_path = "/content/dataset.zip"

# Extract to a specific directory
extract_path = "/content/dataset"

# Create directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")


Extraction complete!


# Getting all the classes

In [ ]:
import pathlib
import numpy as np
data_dir=pathlib.Path("./dataset/Dataset/train")
class_name=np.array(sorted([item.name for item in data_dir.glob("*")]))
print(class_name)

['airplanes' 'cars' 'ship']


# Resizing and turn into batches data

In [ ]:
# preporcessing data
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255)
test_datagen=ImageDataGenerator(rescale=1./255)

train_dir = "./dataset/Dataset/train/"
test_dir = "./dataset/Dataset/test/"

# import data from directories and turn it into batches
train_data=train_datagen.flow_from_directory(train_dir,
                                             batch_size=32,
                                             target_size=(224, 224),
                                             class_mode="categorical",
                                             seed=42)
test_data = test_datagen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)

Found 3000 images belonging to 3 classes.
Found 582 images belonging to 3 classes.


# Define and compile the CNN Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Softmax for multiclass classification
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',  # Use sparse_categorical_crossentropy if labels are integers
              metrics=['accuracy'])

# Summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,347 (42.61 MB)

 Trainable params: 11,169,347 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

# Fitting model

In [ ]:
history = model.fit(train_data,
                    validation_data=test_data,
                    epochs=30,
                    batch_size=32
                    )

Epoch 1/30
94/94 ━━━━━━━━━━━━━━━━━━━━ 33s 290ms/step - accuracy: 0.5622 - loss: 1.3079 - val_accuracy: 0.8677 - val_loss: 0.3656
Epoch 2/30
94/94 ━━━━━━━━━━━━━━━━━━━━ 23s 244ms/step - accuracy: 0.8566 - loss: 0.4092 - val_accuracy: 0.9003 - val_loss: 0.2846
Epoch 3/30
94/94 ━━━━━━━━━━━━━━━━━━━━ 24s 260ms/step - accuracy: 0.9004 - loss: 0.2742 - val_accuracy: 0.9107 - val_loss: 0.2699
Epoch 4/30
94/94 ━━━━━━━━━━━━━━━━━━━━ 22s 230ms/step - accuracy: 0.9339 - loss: 0.1998 - val_accuracy: 0.9021 - val_loss: 0.3026
Epoch 5/30
94/94 ━━━━━━━━━━━━━━━━━━━━ 26s 274ms/step - accuracy: 0.9356 - loss: 0.1725 - val_accuracy: 0.9210 - val_loss: 0.2340
Epoch 6/30
94/94 ━━━━━━━━━━━━━━━━━━━━ 24s 254ms/step - accuracy: 0.9482 - loss: 0.1336 - val_accuracy: 0.9210 - val_loss: 0.2822
Epoch 7/30
94/94 ━━━━━━━━━━━━━━━━━━━━ 20s 208ms/step - accuracy: 0.9657 - loss: 0.0879 - val_accuracy: 0.9089 - val_loss: 0.2817
Epoch 8/30
94/94 ━━━━━━━━━━━━━━━━━━━━ 22s 229ms/step - accuracy: 0.9624 - loss: 0.0942 - val_accu

# Evaluate the Model

In [ ]:
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.2f}")

19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 155ms/step - accuracy: 0.9099 - loss: 0.6258
Test Accuracy: 0.91


# Save and Use the Model

In [ ]:
model.save("multiclass_cnn_model(Car, Bus or Ship).h5")